In [1]:
#https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering

# 1. Import Dependencies

In [2]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Types of Spaces

In [3]:
Discrete(3)

Discrete(3)

In [4]:
Box(0,1,shape=(3,3)).sample()

array([[0.23478743, 0.83170736, 0.01341087],
       [0.6111086 , 0.96186954, 0.22686833],
       [0.08431793, 0.9945125 , 0.4011001 ]], dtype=float32)

In [5]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[ 87, 138,  85],
       [ 80,  71, 229],
       [ 18,  55, 215]])

In [6]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

(1, array([76.06112], dtype=float32))

In [7]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([87.747536], dtype=float32))])

In [8]:
MultiBinary(4).sample()

array([0, 0, 1, 0], dtype=int8)

In [9]:
MultiDiscrete([5,2,2]).sample()

array([1, 1, 0])

# 3. Building an Environment

In [10]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [11]:
env=ShowerEnv()

/usr/local/anaconda3/envs/bmrl/lib/python3.7/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [12]:
env.observation_space.sample()

array([58.025944], dtype=float32)

In [13]:
env.reset()

array([35.])

In [14]:
from stable_baselines3.common.env_checker import check_env

In [15]:
check_env(env, warn=True)

# 4. Test Environment

In [16]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-40
Episode:2 Score:-28
Episode:3 Score:-14
Episode:4 Score:-12
Episode:5 Score:-40


In [17]:
env.close()

# 5. Train Model

In [18]:
log_path = os.path.join('Training', 'Logs')

In [19]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [20]:
model.learn(total_timesteps=400000)

Logging to Training/Logs/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -28.2    |
| time/              |          |
|    fps             | 2930     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -25.3       |
| time/                   |             |
|    fps                  | 1893        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008111389 |
|    clip_fraction        | 0.0522      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0.000277    |

KeyboardInterrupt: 

# 6. Save Model

In [ ]:
model.save('PPO')

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)